In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import *

In [3]:
dataset = 'config/dataset/clean_dataset_08.02.2022.yaml'

with open(dataset, 'r') as stream:
    files = yaml.safe_load(stream)

In [6]:
csv_header = 'ID;Licence plate;Country;LPM confidence;OCR confidence;Frame count;ROI;Direction;Start time;End time;Best detection X-coordinate;Best detection Y-coordinate;Best detection corners;Best detection frame number;Best detection frame timestamp;Category;Category confidence;Color;Color confidence;Make;Make confidence;Model;Model confidence;View;View confidence;Source file name;Image file name'

interval_dataset = []

for file in files:
    intervals = load_intervals(file)

    csv = load_csv(file)
    csv = csv.astype(str)

    events_start_time, events_end_time = load_event_time_from_csv(csv)

    audio, sr = load_audio(file, return_sr=True)

    events = load_events(file)
    
    # '''
    for interval_from, interval_till in intervals:
        interval_audio = audio[int(interval_from * sr) : int(interval_till * sr)]

        mask = (interval_from <= events_start_time) & (interval_till > events_end_time)
        interval_csv = csv[mask]

        interval_file = f'{file}_{interval_from:.0f}_{interval_till:.0f}'
        interval_dataset.append(interval_file)

        # create offset
        s, ms = divmod(interval_from, 1)
        m, s = divmod(s, 60)
        h, m = divmod(m, 60)
        offset_str = f'{h:.0f}:{m:.0f}:{s:.0f}.{ms * 10:.0f}'
        offset = datetime.strptime(f'{h:.0f}:{m:.0f}:{s:.0f}.{ms * 10:.0f}', '%H:%M:%S.%f')

        # apply offset to columns
        def apply_offset(column):
            start_time = interval_csv[:, column]
            for i, t in enumerate(start_time):
                interval_csv[i, column] = datetime.strptime(t, '%H:%M:%S.%f') - offset

        apply_offset(CsvColumnID.START_TIME)
        apply_offset(CsvColumnID.END_TIME)
        apply_offset(CsvColumnID.BEST_DETECTION_FRAME_TIME)

        # apply offset to labels
        mask = (interval_from <= events) & (interval_till > events)
        interval_events = events[mask]
        interval_events = np.round(interval_events - interval_from, 2)

        # torch.save([interval_audio, sr], f'data/audio_tensors/cvut_intervals/{interval_file}.pt')
        np.savetxt(f'data/csv/cvut_intervals/{interval_file}.csv', interval_csv, fmt='%s', delimiter=';', header=csv_header)
        np.savetxt(f'data/labels/cvut_intervals/{interval_file}.txt', interval_events, fmt='%s')


20190819-Kutna Hora-L3-out-MVI_0008_0_5
20190819-Ricany-L8-out-MVI_0013_2053_2062


In [9]:
from datetime import datetime

In [8]:
name = '20190819-Kutna Hora-L1-out-MVI_0007'

folder='data/csv/cvut_intervals/*.csv'
glob(folder, recursive=True)
for file in glob(folder, recursive=True):
    if file.find(name) != -1:

        print(file)

data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_831_890.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1396_1567.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_129_210.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_55_129.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1154_1197.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_247_367.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1782_1805.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1887_1913.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1226_1396.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1935_2067.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_367_440.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_690_778.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-MVI_0007_1567_1623.csv
data/csv/cvut_intervals/20190819-Kutna Hora-L1-out-

In [10]:
n_folds = 9

dataset_from = '25.03.2022'
dataset_to = '29.03.2022'

for stage in ['validation_files', 'testing_files', 'training_files']:
    os.makedirs(f'config/{stage}/{dataset_to}', exist_ok=True)

    for fold in range(n_folds):
        dataset_from_path = f'config/{stage}/{dataset_from}/{fold}.yaml'
        dataset_to_path = f'config/{stage}/{dataset_to}/{fold}.yaml'

        if stage == 'testing_files' or stage == 'training_files':
            shutil.copy2(dataset_from_path, dataset_to_path)
        else:
            with open(dataset_from_path, 'r') as stream:
                files = yaml.safe_load(stream)

            F = []
            
            for name in interval_dataset:
                for file in files:
                    if name.find(file) != -1:
                        F.append(f'- cvut_intervals/{name}')
            
            np.savetxt(dataset_to_path, F, fmt='%s')

['- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_0_713',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_713_903',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_903_1118',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_1118_1412',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_1412_1620',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_1620_1789',
 '- cvut_intervals/20190819-Kutna Hora-L13-in-MVI_0006_1789_2042',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_0_29',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_29_243',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_243_262',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_262_375',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_375_478',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_478_511',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_511_601',
 '- cvut_intervals/20190819-Kutna Hora-L14-out-MVI_0005_601_631',
 '- cvut_inter

In [6]:
load_audio('cvut_intervals/20190819-Ricany-L6-out-MVI_0011_0_379')

tensor([0.0279, 0.0766, 0.1262,  ..., 0.0258, 0.0179, 0.0104])